In [8]:
import pandas as pd
import numpy as np
import sys,os
this_dir = os.path.abspath('.')
parent_dir = os.path.abspath('..')
if this_dir not in sys.path:
    sys.path.append(parent_dir)
    sys.path.append(this_dir)
import db_info#@UnresolvedImport
import typing


In [56]:
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'

In [58]:
StrList = typing.List[str]
class SqlDownloader():
    def __init__(self,config_name:str='local'):
        self.pga = db_info.get_db_info(config_name=config_name)
    
    def get_sql(
        self,
        columns=None,
        table=futtab,
        where_clauses=None,
        groupby_columns=None,
        orderby_columns=None,
        limit=None
    ):
        cols = '*' if columns is None else ','.join(columns)
        sql = f"select {cols} from {table}"
        
        if where_clauses is not None:
            wc_string = f'where {where_clauses[0]}'
            if len(where_clauses)>1:
                wc_string =  wc_string + " and " + ' and '.join(where_clauses[1:])
            wc_string = wc_string.replace('%','%%')
            sql = sql + " " + f"{wc_string}"
        if groupby_columns is not None:
            sql += f" group by {','.join(groupby_columns)}"
        if orderby_columns is not None:
            sql += f" order by {','.join(orderby_columns)}"
        if limit is not None:
            sql += f" limit {int(str(limit))}"
        df = self.pga.get_sql(sql)
        return df
        
    def get_options(self,symbol:str,yyyymmdd_beg:int=None,yyyymmdd_end:int=None):
        sql = f"select * from {opttab} where symbol='{symbol}'"
        if yyyymmdd_beg is not None:
            sql += f' and settle_date>={yyyymmdd_beg}'
        if yyyymmdd_end is not None:
            sql += f' and settle_date<={yyyymmdd_end}'
        df =  self.pga.get_sql(sql)
        if df is not None and len(df)>2:
            df = df.sort_values(['pc','settle_date','strike'])
        return df

    def get_futures(self,symbol_list:StrList,yyyymmdd_beg:int=None,yyyymmdd_end:int=None):
        if type(symbol_list) is list or type(symbol_list) is np.ndarray:
            sym_string = ",".join([f"'{s}'" for s in symbol_list]) 
        else:
            sym_string = f"'{symbol_list}'"
        sql = f"select * from {futtab} where symbol in ({sym_string})"
        if yyyymmdd_beg is not None:
            sql += f' and settle_date>={yyyymmdd_beg}'
        if yyyymmdd_end is not None:
            sql += f' and settle_date<={yyyymmdd_end}'
        df =  self.pga.get_sql(sql)
        if df is not None and len(df)>2:
            df = df.sort_values(['symbol','settle_date'])
        return df


In [59]:
sqld = SqlDownloader(config_name='local')

  sec_db


In [19]:
sqld.get_futures('CLF19')

,symbol,settle_date,contract_num,open,high,low,close,adj_close,volume,open_interest
0,CLF19,20131120,62,0.00,80.35,80.35,80.35,80.35,0,0
1,CLF19,20131121,61,0.00,79.91,79.91,79.91,79.91,0,0
2,CLF19,20131122,61,0.00,79.95,79.95,79.95,79.95,0,0
3,CLF19,20131125,61,0.00,80.35,80.35,80.35,80.35,0,0
4,CLF19,20131126,61,0.00,80.51,80.51,80.51,80.51,0,0
...,...,...,...,...,...,...,...,...,...,...
1274,CLF19,20181213,1,51.20,53.27,50.35,52.58,52.58,730810,186783
1275,CLF19,20181214,1,52.83,52.95,50.84,51.20,51.20,733627,146352
1276,CLF19,20181217,1,51.25,51.87,49.01,49.88,49.88,526839,90329
1277,CLF19,20181218,1,49.16,49.59,45.79,46.24,46.24,211257,66901


In [20]:
sqld.get_options('CLF19')

,symbol,strike,pc,settle_date,open,high,low,close,adj_close,volume,open_interest
0,CLF19,50.0,C,20160421,8.4,8.40,8.08,8.08,8.08,0,0
1,CLF19,50.0,C,20160422,0.0,8.45,8.45,8.45,8.45,90,90
2,CLF19,50.0,C,20160425,0.0,8.36,8.36,8.36,8.36,0,90
3,CLF19,50.0,C,20160426,0.0,8.81,8.81,8.81,8.81,0,90
4,CLF19,50.0,C,20160427,0.0,9.50,9.50,9.50,9.50,0,90
...,...,...,...,...,...,...,...,...,...,...,...
33568,CLF19,91.0,P,20181214,0.0,39.80,39.80,39.80,39.80,0,0
33571,CLF19,92.0,P,20181214,0.0,40.80,40.80,40.80,40.80,0,0
33573,CLF19,92.5,P,20181214,0.0,41.30,41.30,41.30,41.30,0,0
33575,CLF19,93.0,P,20181214,0.0,41.80,41.80,41.80,41.80,0,0


In [54]:
wc_list = [
    "symbol='CLF19'",
    "pc='C'",
    "strike=50",
    "settle_date>20180501"
]
sqld.get_sql(table=opttab,where_clauses=wc_list)

,symbol,strike,pc,settle_date,open,high,low,close,adj_close,volume,open_interest
0,CLF19,50.0,C,20180502,0.00,15.43,15.43,15.43,15.43,0,98
1,CLF19,50.0,C,20180503,15.35,15.81,15.35,15.81,15.81,0,98
2,CLF19,50.0,C,20180504,0.00,16.99,16.99,16.99,16.99,2,98
3,CLF19,50.0,C,20180507,0.00,18.05,18.05,18.05,18.05,0,98
4,CLF19,50.0,C,20180508,17.40,17.40,16.71,16.71,16.71,0,98
...,...,...,...,...,...,...,...,...,...,...,...
155,CLF19,50.0,C,20181210,3.15,3.15,1.67,1.82,1.82,282,3452
156,CLF19,50.0,C,20181211,1.82,2.59,1.65,2.04,2.04,244,3566
157,CLF19,50.0,C,20181212,2.51,2.86,1.44,1.47,1.47,105,3583
158,CLF19,50.0,C,20181213,1.57,3.20,1.05,2.64,2.64,437,3389


In [60]:
cols = ['distinct symbol']
wc_list = [
    "symbol like 'CL%'",
]
sqld.get_sql(table=futtab,columns=cols,where_clauses=wc_list)

,symbol
0,CLM17
1,CLN11
2,CLV20
3,CLG15
4,CLK15
...,...
238,CLK22
239,CLN18
240,CLN13
241,CLK25


In [4]:
#!jupyter nbconvert secdb_example.ipynb --to python